In [8]:
#Name: Abel
#Date: 2021.08.10
#Purose: To prepare the excel spread sheet to be ML ready
#

In [9]:
import os
from openpyxl import load_workbook


def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    @param filename: File path or existing ExcelWriter
                     (Example: '/path/to/file.xlsx')
    @param df: DataFrame to save to workbook
    @param sheet_name: Name of sheet which will contain DataFrame.
                       (default: 'Sheet1')
    @param startrow: upper left cell row to dump data frame.
                     Per default (startrow=None) calculate the last row
                     in the existing DF and write to the next row...
    @param truncate_sheet: truncate (remove and recreate) [sheet_name]
                           before writing DataFrame to Excel file
    @param to_excel_kwargs: arguments which will be passed to `DataFrame.to_excel()`
                            [can be a dictionary]
    @return: None

    Usage examples:

    >>> append_df_to_excel('d:/temp/test.xlsx', df)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, header=None, index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2',
                           index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2', 
                           index=False, startrow=25)

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    # Excel file doesn't exist - saving and exiting
    if not os.path.isfile(filename):
        df.to_excel(
            filename,
            sheet_name=sheet_name, 
            startrow=startrow if startrow is not None else 0, 
            **to_excel_kwargs)
        return
    
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')

    # try to open an existing workbook
    writer.book = load_workbook(filename)
    
    # get the last row in the existing Excel sheet
    # if it was not specified explicitly
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    # truncate sheet
    if truncate_sheet and sheet_name in writer.book.sheetnames:
        # index of [sheet_name] sheet
        idx = writer.book.sheetnames.index(sheet_name)
        # remove [sheet_name]
        writer.book.remove(writer.book.worksheets[idx])
        # create an empty sheet [sheet_name] using old index
        writer.book.create_sheet(sheet_name, idx)
    
    # copy existing sheets
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()
    

def modelClean (inp):  # Isolates only the model of the vehicle
    models = ['LX', 'EX', 'SI','Sport Touring', 'Sport', 'Touring', 'SE']
    #inp = "Civic Sedan LX"
    tempSpl = inp.replace('|', ' ').split()
    for x in models:
        for y in tempSpl:
            if y.find(x) != -1: #this is necessary to account for sub-trims (EX-T)
                return y
    return 'N/A'

def yearClean (inp): #for year, mileage, and price
    if isinstance(inp, int): #if it's already an int, simply return the int
        return inp
    tempSpl = inp.split()
    for x in tempSpl: #returns the int value (if there are other strings attached)
        try:
            return int(x)
        except:
            continue
    return 'N/A'

def numOnly(inp): #inp is a string
    if isinstance(inp, int) or isinstance(inp, float): #if it's already an int, simply return the int
        return inp
    output=''
    for x in inp:    
        try:
            int (x)
            output += x
        except:
            if x == '.': # if it's a decimal, cut if off
                break
            continue
    if output == '':
        return inp
    else:
        return output
    

def mileageClean (inp):
    try:
        return int(inp.lower().replace('km', '').replace(',', ''))
    except: #if it's already an int or contains no int, simply return the input
        return inp

def priceClean (inp):
    try:
        return int(inp.lower().replace('$', '').replace('*', '').replace(',', '').replace('\n', '').strip())
    except: #if it's already an int or contains no int, simply return the input
        return inp
    
def colorClean (inp): 
    colors = ['blue', 'black', 'red', 'silver', 'white', 'brown', 'green', 'yellow', 'grey', 'gray', 'steel']
    grey = ['gray', 'steel']
    #inp = "Civic Sedan LX"
    try:
        tempSpl = inp.split()
        for x in tempSpl:
            if x.lower() in colors:
                if x.lower() in grey:
                    return 'grey'
                else:
                    return x.lower()
    except:
        return 'N/A'    
    return inp


In [10]:
import pandas as pd
import xlwings as xw
import openpyxl

filename = '2021.08.10 Honda Civics.xlsx'
wb = openpyxl.load_workbook(filename)
for x in wb.sheetnames:
    df = pd.read_excel(filename, sheet_name=x)
    #print (x) #Sheetname
    #df['Model'] = map(modelClean(), df['Model'])
    df['Vehicle'] = df['Vehicle'].apply (modelClean)
    df['Year'] = df['Year'].apply (numOnly)
    df['Price'] = df['Price'].apply (numOnly)
    df['Mileage'] = df['Mileage'].apply (numOnly)
    try:
         df['Color'] = df['Color'].apply (colorClean)
    except:
        print ('Missing Color here')
    df.loc[:, 'Location'] = x
    df = df.drop(columns="Link").drop(columns="Carfax Link")
    #print(df.head(10))
    #print (df.shape)
    append_df_to_excel (filename[:-5] + ' (MLReady).xlsx', df, sheet_name=x, index=False)
